### Tools

In [1]:
%pip install -U langchain-community langgraph langchain-openai tavily-python

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.4 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

import getpass
import os
# load env file
from dotenv import load_dotenv
load_dotenv()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

True

In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Los Angeles")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

/Users/ttran/PyCharmProjects/aiug-workship-vs/aiug-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Los Angeles', 'region': 'California', 'country': 'United States of America', 'lat': 34.05, 'lon': -118.24, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722452903, 'localtime': '2024-07-31 12:08'}, 'current': {'last_updated_epoch': 1722452400, 'last_updated': '2024-07-31 12:00', 'temp_c': 28.6, 'temp_f': 83.5, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 219, 'wind_dir': 'SW', 'pressure_mb': 1013.0, 'pressure_in': 29.92, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 33, 'cloud': 0, 'feelslike_c': 27.9, 'feelslike_f': 82.3, 'windchill_c': 28.6, 'windchill_f': 83.5, 'heatindex_c': 27.9, 'heatindex_f': 82.3, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 8.5, 'gust_kph': 13.7}}"}, {'url': 'https://www.easeweather.com/north-america/united-

### Bind model with tools

In [4]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# initialize model
model = ChatOpenAI(model="gpt-4o")

In [6]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [7]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [8]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [9]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in San Francisco?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_HLz4dhEVBlFjD1Neoy1J9RhU', 'type': 'tool_call'}]


### Using a prebuilt react agent with tools

In [10]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools) # simpify the prompting techniques with tools

In [11]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='18def95d-e530-48ec-9ac1-2f1398b726b0'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-85295a8b-4b09-40a9-ab21-7c98e6226016-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [12]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in San Francisco?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in San Francisco?", id='774b1285-f62b-4680-9f0a-1f39456bb060'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PxiIeYqdb7izBnl6oixG28zT', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e7904486-6f2c-4053-b94a-f568fef83aea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_PxiIeYqdb7izBnl6oixG28zT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'Cali